In [2]:
import numpy as np
import pandas as pd

In [3]:
# reference code for generating 3d plots by index.
# # prepare some coordinates
# x, y, z = np.indices((8, 8, 1))

# # draw cuboids in the top left and bottom right corners, and a link between
# # them
# cube1 = (x < 3) & (y < 3) & (z < 1)
# cube2 = (x >= 5) & (y >= 5) & (z <1)
# #link = abs(x - y) + abs(y - z) + abs(z - x) <= 2

# # combine the objects into a single boolean array
# voxelarray = cube1 | cube2 #| link

# # set the colors of each object
# colors = np.empty(voxelarray.shape, dtype=object)
# #colors[link] = 'red'
# colors[cube1] = 'blue'
# colors[cube2] = 'green'

# # and plot everything
# ax = plt.figure().add_subplot(projection='3d')
# ax.voxels(voxelarray, facecolors=colors, edgecolor='k')

# plt.show()

In [4]:
#refer to cat pieces by colours. try not to forget em. if in doubt, ROY G BIV lightest hue to darkest

#lets exclude the third dimension to start with because that shit is hard and just encode these as the minimal size 
#binary matrix that can surround a given piece.
#generate containing array of 1, then remove gaps in rectangle
#sike, doing it as squares because it makes matrix transformations/uniqueness checks work better.

#let's start this all in two dimensions, which is probably gonna be a pain later, but should make this a lot easier.
#also, going to store these in a dictionary for indicing/iteration purposes

cat_states=dict()

white=np.full((3,3), "W", dtype=str)
white[0] = ""
white[1][1:] = ""
cat_states['white']=white

pink=np.full((3,3),"P", dtype=str)
pink[0][1:] = ""
pink[2][1]=""
cat_states['pink']=pink

red=np.full((4,4),"R", dtype=str)
red[0]=""
red[1][1:] = ""
red[3][1:3]=""
cat_states['red']=red

yellow=np.full((3,3),"Y", dtype=str)
yellow[0][1:] = ""
cat_states['yellow']=yellow

mint=np.full((3,3), "M", dtype=str)
mint[0][0] = ""
mint[0][2] =""
mint[1][0] = ""
cat_states['mint']=mint

green=np.full((4,4),"G", dtype=str)
green[0][0:2] = ""
green[1][1:3] = ""
green[3][1:3] = ""
cat_states['green']=green

arctic=np.full((4,4), "A", dtype=str)
arctic[0]=""
arctic[1][0:3]=""
cat_states['arctic']=arctic

sky=np.full((2,2), "S", dtype=str)
sky[0]=""
cat_states['sky']=sky

teal=np.full((2,2), "T", dtype=str)
teal[0][1]=""
cat_states['teal']=teal

indigo=np.full((2,2), "I", dtype=str)
cat_states['indigo']=indigo

violet=np.full((3,3), "V", dtype=str)
violet[0]=""
cat_states['violet']=violet

black=np.full((3,3), "B", dtype=str)
black[0][1]=""
cat_states['black']=black


In [5]:
def _generate_rotations(cat: np.array)-> np.array:
    """Generates four rotation states of cat piece from initial state.

    Args:
        cat (np.array): Numpy array representing cat shape

    Returns:
        list[np.array]: Rotation states of of shape
    """
    states=[cat]
    for i in range(3):
        #rotate clockwise and add to state list
        cat=np.rot90(cat)
        states.append(cat)
    #remove any duplicates.
    states=np.unique(states, axis=0)
    return states

def _generate_states(cat: np.array)-> np.array:
    """Generates up to 8 rotation states of cats -- rotations, flip then rotations, then removes any duplicates.

    Args:
        cat (np.array): Numpy array representing cat shape.

    Returns:
        np.array: All orientations of shape induced by rotation and flip
    """
    
    #get rotation states
    states=_generate_rotations(cat)
    #get flip states
    f_states= _generate_rotations(np.fliplr(cat))

    #concat and remove duplicates
    unique_states=np.unique(np.vstack([states, f_states]), axis=0)
    return unique_states


In [6]:
#dictionary of all shape states

for key, val in cat_states.items():
    cat_states[key]= _generate_states(val)

In [7]:
#awkward part: these arrays might have a 0 row or column because they're squares to keep array dimensions consistent across rotations.
#we could take these out and make them lists of different dimensioned arrays.....but i don't feel like figuring that out right now
#so instead, for a puzzle on an m x n grid, we're going to initiate an (m+2) x (n+2) grid where the outside rectangle will be a fixed border.

#any border/missing spaces will be set to value "X"

def init_puzzle_grid(max_height, max_width, exceptions=False):
    grid= np.full((max_height+2, max_width+2), "", dtype=str)
    grid[0]="X"
    grid[-1] = "X"
    grid[:,0]="X"
    grid[:, -1] ="X"

    return grid

In [8]:
cat_priority={"green":1,
              "arctic":2,
              "red":3,
              "black":4,
              "yellow":5,
              "mint" : 6,
              "pink": 7,
              "violet": 8,
              "white": 9,
              "indigo": 10,
              "teal": 11,
              "sky": 12}


In [9]:
#conventions --- refer to grid positions by matrix index notation (0,0) is top left.
#record tried states/board state by:
#(cat, cat_state_index, i, j) where
# cat: semantic name of cat
# cat_state_index: numerical list index of the orientation in cat_states[cat] being tried
# i,j row/col index where (0,0) cord of cat_states[cat][cat_state_index] is placed.

#this is gonna end up being recursive, isn't it.

In [10]:
def _get_next_cat(cats: list, priorities=cat_priority):
    prior_list= [priorities[cat] for cat in cats]
    return prior_list
def _check_solved(state: np.array) -> bool:
    #return false if any empty strings in puzzle state
    return np.all(state != "")
def _is_valid_grid(grid):
    #just checks if all strings in grid are 1 character or less. if anything more, means we've overlapped somewhere.
    return (np.strings.str_len(grid) <= 1).all()
        

In [41]:
#using these to hold state for a given orientation as we iterate thru possible placements for it in a puzzle
#this is probably dumb
#this is definitely ugly.
#not sure cat_states needs to be passed through all of this but losing track of what scope i need to call it in.
class Cat_Placement:
    def __init__(self, colour, grid_init, i = -1, j= -1, orientation_idx=0, cat_states=cat_states):
        self.colour = colour
        self.orientation_idx = orientation_idx
        self.orientations = cat_states[colour]
        self.orientation = self.orientations[orientation_idx]
        self.cat_size= self.orientation.shape[0]
        self.i=i
        self.j=j
        self.make_cat_placement_index_gen(grid_init)

    def make_cat_placement_index_gen(self, grid_init):
        grid_height = grid_init.shape[0]
        grid_width = grid_init.shape[1]
        i_max=grid_height-self.cat_size
        j_max=grid_width - self.cat_size
        o_max = len(self.orientations) - 1
        
        def idx_gen_func(self, i_max=i_max, j_max=j_max, o_max=o_max):
            i, j, o = 0,0,0 #col, row, orientation index
            while True:
                self.i = i
                self.j= j
                self.orientation_idx = o
                yield i, j, o
                #increment column position
                #print("incrementing column index")
                j += 1
                if j >j_max:
                    #if at max columns, return to 0 indexed column and move one row down
                    #print("exceeded column count")
                    j=0
                    i+=1
                if i > i_max:
                    # if at max rows, return to 0,0 and try next orientation
                   # print("exceeded rowcount")
                    i = 0
                    j= 0 
                    #print("updating orientation index")
                    o +=1
                    #update orientation
                    #print("updating self orientation")
                    self.orientation = self.orientations[o]
                   # print("self orientation update succeded.")
                if o > o_max:
                    raise Exception("All indices exhausted.")
        self.idx_gen = idx_gen_func(self)

In [43]:
def _place_o_at_idx(grid_state, o_plc):
    #make placement array
    #returns results of placement regardless if valid or not
    i = o_plc.i
    j = o_plc.j
    cat_size= o_plc.cat_size
    orientation=o_plc.orientation
    placement = grid_state[i: i + cat_size, j: j+ cat_size] + orientation
    return placement

def place_orientation(grid_state, o_plc):
    #trigger next placement index and try to place.w
    #loop terminates when it finds something to place
    #otherwise throws exception (in index generator, this might be dumb, i don't know shit about exception handling)
    #upon running out of indices to "next" to.

    #AT SOME POINT, NEEDS REAL EXCEPTION HANDLING SO THAT IF THIS ERRORS, WE REVERT PLACEMENT OF PREVIOUS PIECE.
    while True:
        #print("trying next index")
        next(o_plc.idx_gen)
        #print("next index sucessfully retrieved")
        #print("trying to place")
        placement= _place_o_at_idx(grid_state, o_plc)
        #print("placed without error")
        #print(placement)
        if _is_valid_grid(placement):
            #if so, update grid state and return it
            #print("placement was valid")
            print(placement)
            grid_state[o_plc.i: o_plc.i + o_plc.cat_size, o_plc.j: o_plc.j+ o_plc.cat_size] = placement
            break
    return grid_state


In [31]:
def place_cat(grid_state, cat_col, cat_states=cat_states):
    #for some grid state and some cat, add cat to first possible open state (if possible)
    #get orientations
    #initate orientation placement object
    o_plc=Cat_Placement(cat_col, grid_state, cat_states=cat_states)
    try:
        #try to place o_plc at it's current o,i,j index
        grid_state=place_orientation(grid_state, o_plc)
    except:
        #print(f"Couldn't place {cat_col} orientation {o_idx}")
        #if orientation not final one in orientation list, try again with the next index.
        #if all orientations exhausted, return exception that triggers reverting piece.
        #probably better way  to do this with exceptions instead of this if check.

        print(f"Couldn't place {cat_col} token")
        raise Exception(f"Couldn't place {cat_col} token")

    return grid_state, o_plc

In [32]:
def solve_puzzle(cats: list[str], grid_init: np.array, symmetry=True):
    #sort cats by priority
    #doing this in reverse order so that we can use .pop to get the highest priority piece.
    cats.sort(key = lambda x: -cat_priority[x])
    to_place= cats.copy()
    placed_cats=[]
    grid_state=grid_init.copy()
    
    #place first cat. special case:
    #if symmetry=True, (m x n) puzzle grid has horizontal/vertical symmetry
    #this means while searching solutions we can hold the inital cat to be placed within
    #the (ceiling(m/2), ceiling(n/2)) top left subgrid.

    #next case: place it wherever and brute force. real high level stuff.

    #this will never be true first time around, so we enter puzzle solving routine
    while len(to_place) > 0:
        #get highest priority piece
        next_cat=to_place[-1]
        print(f"Placing {next_cat}")
        try:
            grid_state, placed_cat=place_cat(grid_state, next_cat)
        except:
            #revert previous piece and try again.
            
            #TEMP: remove from to_place anyways to ensure this doesn't infinite loop
            to_place.pop()
        else:
            #cat successfully placed
            #remove cat from to_place
            to_place.pop()
            #log placement to placed
            placed_cats.append(placed_cat)
        print(grid_state)
      

    return grid_state, placed_cats


In [16]:
cats = ['black', 'white', 'red']
cats.sort(key = lambda x: -cat_priority[x])

In [44]:
grid1=init_puzzle_grid(5,5)
grid, cats=solve_puzzle(['black', 'mint', 'violet', 'sky', 'teal'], grid1)

Placing black
[['B' '' 'B']
 ['B' 'B' 'B']
 ['B' 'B' 'B']]
[['X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'B' '' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' '' '' 'X']
 ['X' '' '' '' '' '' 'X']
 ['X' '' '' '' '' '' 'X']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X']]
Placing mint
[['B' 'B' 'M']
 ['M' 'M' 'M']
 ['' 'M' 'M']]
[['X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'B' '' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' 'M' '' 'X']
 ['X' '' 'M' 'M' 'M' '' 'X']
 ['X' '' '' 'M' 'M' '' 'X']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X']]
Placing violet
Couldn't place violet token
[['X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'B' '' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' '' '' 'X']
 ['X' 'B' 'B' 'B' 'M' '' 'X']
 ['X' '' 'M' 'M' 'M' '' 'X']
 ['X' '' '' 'M' 'M' '' 'X']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X']]
Placing teal
[['' 'T']
 ['T' 'T']]
[['X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'B' '' 'B' '' 'T' 'X']
 ['X' 'B' 'B' 'B' 'T' 'T' 'X']
 ['X' 'B' 'B' 'B' 'M' '' 'X']
 ['X' '' 'M' 'M' 'M' '' 'X']
 ['X' '' '' 'M' 'M' '' 'X']
 ['X' 'X' 'X

In [17]:
cats